In [1]:
import os
import sys
from pathlib import Path # if you haven't already done so
file = Path(os.getcwd()).resolve()
parent, root = file.parent, file.parents[1]
print(parent, root)
sys.path.append(str(parent))
# sys.path.append("..")

# # Additionally remove the current file's directory from sys.path
# try:
#     sys.path.remove(str(parent))
# except ValueError: # Already removed
#     pass

/root/fruitao/drone/logic_guided_prompt/code /root/fruitao/drone/logic_guided_prompt


In [2]:
from drone_train import *

/root/fruitao/drone/logic_guided_prompt/code/data_augmentation /root/fruitao/drone/logic_guided_prompt/code


/root/miniconda3/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
dataset_file_li=[r"../../text_annotation/desert.xlsx", r"../../text_annotation/disturbance.xlsx", r"../../text_annotation/factory.xlsx", r"../../text_annotation/island.xlsx", 
                 r"../../text_annotation/misc.xlsx", r"../../text_annotation/ocean.xlsx", r"../../text_annotation/rural.xlsx", r"../../text_annotation/urban.xlsx"
]

In [4]:
old_data_dic=load_sensor_data(dataset_file_li)

templates:{'flying_not_normal_frame': ['The drone is flying with a damaged frame.'], 'near_moving_object': ["(0:01) The drone is risking physical damage. It's flying too close to the moving car at a distance of 3.0m.", "(0:05) The drone is risking physical damage. It's flying too close to the moving car at a distance of 0.3m."], 'reachable_inpath': ["(0:05) The drone is facing the risk of physical damage. There is a car in the drone's flight path at a distance of 0.3m."]}
templates:{'near_moving_object': ["(0:01) The drone is risking physical damage. It's flying too close to the moving car at a distance of 3.0m.", "(0:03) The drone is risking physical damage. It's flying too close to the moving car at a distance of 0.2m."], 'reachable_inpath': ["(0:03) The drone is facing the risk of physical damage. There is a car in the drone's flight path at a distance of 0.2m."]}
templates:{'flying_not_normal_frame': ['The drone is flying with a damaged frame.'], 'near_moving_object': ["(0:03) The 

In [5]:
(text_li, all_status_dic_li, all_timestep_dic_li, all_sensor_data_li), (templates_li, related_sensor_data_li), oneshot_examples_li=read_data_dict(old_data_dic)

In [6]:
print(len(text_li), text_li[0])
print(len(all_status_dic_li), all_status_dic_li[0])
print(len(all_timestep_dic_li), all_timestep_dic_li[0])
print(len(all_sensor_data_li), all_sensor_data_li[0])

300 [{'Text1': "Risk of physical damage! The drone has a damaged frame. [SEP]\n(0:05) Risk of physical damage! There's a car in the drone's flight path at a distance of .3m. [SEP]", 'Text2': "Risk of physical damage! The drone's frame is in need of repair. [SEP]\n(0:05) Risk of physical damage! In .3m there's a car in the flight path. [SEP]"}]
300 [{'WindSpeed (m/s)': 0, 'DroneSpeed (m/s)': 0.5, 'PilotExperienced': True, 'Altitude (m)': 0.5, 'Temperature (celcius)': 41, 'Distance_from_remote_control (m)': 96, 'Battery_level': 90, 'Low_visibility': False, 'Normal_frame': False, 'weather': 'sunny', 'upside_down': False, 'good_motor_condition': True, 'going_backwards': False, 'indoor': False, 'waterproof drone?': False, 'flying over': 'ground', 'criticality': 'warning', 'RiskOfPhysicalDamage': nan, 'RiskOfInternalDamage': False, 'RiskOfHumanDamage': False, 'LostConnection': False}]
300 [{'name': 'car', 'ID_obj': 1.0, 'Type': 'Car', 'Moving': False, 'InPath': False, 'Distance': 3.0, 'time_

In [7]:
print(len(templates_li), templates_li[0])
print(len(related_sensor_data_li), related_sensor_data_li[0])

1652 The drone is flying with a damaged frame.
1652 {"Normal_frame": false}


In [8]:
train_sensor_data, test_sensor_data=train_test_split(related_sensor_data_li)
train_templates, test_templates=train_test_split(templates_li)

# Train

In [9]:
!nvidia-smi

Tue Jun 13 14:23:51 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 515.86.01    Driver Version: 515.86.01    CUDA Version: 11.7     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100 80G...  On   | 00000000:01:00.0 Off |                    0 |
| N/A   67C    P0   236W / 300W |  39641MiB / 81920MiB |     99%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA A100 80G...  On   | 00000000:22:00.0 Off |                    0 |
| N/A   

In [10]:
import torch
import math

from tqdm import tqdm
import numpy as np
from transformers import AutoTokenizer, AutoModelForCausalLM, AutoModel
import torch.nn as nn
from sklearn.metrics import accuracy_score
from nltk.translate import meteor_score

import torch
# try:
#     from transformers import AutoTokenizer, AutoModelForCausalLM, LlamaTokenizer, AutoModel, LlamaForCausalLM
# except ImportError:
#     from transformers import AutoTokenizer, AutoModelForCausalLM, LLaMATokenizer, AutoModel

In [11]:
device=torch.device("cuda:3")
torch.manual_seed(3407)

In [12]:
# n_tokens = 20

model_path= '/root/fruitao/models/models--vicuna-13b'
tokenizer = AutoTokenizer.from_pretrained(model_path, use_fast=False, padding_side="left")
model = AutoModelForCausalLM.from_pretrained(model_path, low_cpu_mem_usage=True)

initialize_from_vocab = True
batch_size = 32
n_epoch = 200

use_ce_loss = False
ce_loss = nn.CrossEntropyLoss()

n_tokens = len(WARNING_DL)
s_wte = SoftEmbedding(model.get_input_embeddings(), 
                      n_tokens=n_tokens, 
                      initialize_from_vocab=initialize_from_vocab)
model.set_input_embeddings(s_wte)

optimizer = torch.optim.Adam(s_wte.parameters(), lr=0.5)

metrics_file='metrics_epoch_vicuna_soft_prompt_200_3407.csv'

from datetime import datetime

now = datetime.now()
dt_string = now.strftime("%d%m_%H%M%S")
save_model_file="./output/vicuna_soft_prompt_200_3407_"+dt_string

if torch.cuda.is_available():
    model = model.to(device)


Loading checkpoint shards: 100%|██████████| 3/3 [00:30<00:00, 10.32s/it]


In [13]:
parameters = list(model.parameters())
for x in parameters[1:]:
    x.requires_grad = False

In [20]:
for x, y, m, dii, true_labels in generate_data(4, n_tokens, related_sensor_data_li, templates_li, tokenizer, device):
    assert dii.shape == y.shape
    # outputs = model.generate(**x)
    outputs = model(input_ids=x, labels=y, attention_mask=m)
    # outputs = model(input_ids=x, labels=y, attention_mask=m, decoder_input_ids=dii)
    assert outputs['logits'].shape[:2] == y.shape
    pred_labels = outputs['logits'][:, :, :].argmax(-1).detach().cpu().numpy()
    print("pred_labels:{}".format((pred_labels, pred_labels.shape)))
    break

ValueError: Expected input batch_size (196) to match target batch_size (244).

In [ ]:
test_losses_epoch, bleu_score_epoch, rouge_score_epoch, meteor_score_epoch=train_with_prompt(model, tokenizer, device, n_epoch=n_epoch, batch_size=batch_size, n_tokens=n_tokens, train_input=train_sensor_data, train_label=train_templates, val_input=test_sensor_data, val_label=test_templates, optimizer=optimizer, ce_loss=ce_loss, use_ce_loss=False)

In [ ]:
(rouge1_precision_epoch, rouge1_recall_epoch, rouge1_fmeasure_epoch), (rouge1_precision_epoch, rouge1_recall_epoch, rouge1_fmeasure_epoch), (rougeL_precision_epoch, rougeL_recall_epoch, rougeL_fmeasure_epoch), (rougeLsum_precision_epoch, rougeLsum_recall_epoch, rougeLsum_fmeasure_epoch), (bleu_avg_score_epoch, bleu_1_score_epoch, bleu_2_score_epoch, bleu_3_score_epoch, bleu_4_score_epoch), (meteor_score_epoch)=save_metrics(test_losses_epoch, bleu_score_epoch, rouge_score_epoch, meteor_score_epoch, metrics_file=metrics_file)

In [ ]:
import matplotlib.pyplot as plt

# https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.plot.html

plt.plot(test_losses_epoch, '*-r')

plt.plot(rouge1_precision_epoch, '*--g')
plt.plot(rouge1_recall_epoch, 'x--g')
plt.plot(rouge1_fmeasure_epoch, 'o--g')

plt.plot(bleu_avg_score_epoch, '*:b')
plt.plot(bleu_1_score_epoch, 'x:b')
plt.plot(bleu_2_score_epoch, '+:b')
plt.plot(bleu_3_score_epoch, 'o:b')
plt.plot(bleu_4_score_epoch, 'd:b')

plt.plot(meteor_score_epoch, '*-.y')

plt.show()

In [ ]:
torch.save(model.state_dict(), save_model_file)

# Metrics

In [22]:
model_path= '/root/fruitao/models/models--vicuna-13b'
tokenizer = AutoTokenizer.from_pretrained(model_path, use_fast=False, padding_side="left")
model = AutoModelForCausalLM.from_pretrained(model_path, low_cpu_mem_usage=True)
n_tokens = len(WARNING_DL)
s_wte = SoftEmbedding(model.get_input_embeddings(), 
                      n_tokens=n_tokens, 
                      initialize_from_vocab=initialize_from_vocab)
model.set_input_embeddings(s_wte)

In [23]:
model.load_state_dict(torch.load("./output/LGP_soft_prompt_200_3407_1306_021205"))
if torch.cuda.is_available():
    model = model.to(device)

In [24]:
def eval_with_prompt(model, tokenizer, device, batch_size, n_tokens,
                     val_input, val_label, optimizer, ce_loss, use_ce_loss=False):
    dev_total_batch = math.ceil(len(val_input) / batch_size)
    
    all_true_labels = []
    all_pred_labels = []
    output_max_pred=[]
    test_losses = []
        
    with torch.no_grad():
            pbar = tqdm(enumerate(generate_data(batch_size, n_tokens, val_input, val_label, tokenizer, device)), total=dev_total_batch)
            for i, (x, y, m, dii, true_labels) in pbar:
                
                all_true_labels += true_labels
                outputs = model(input_ids=x, labels=y, attention_mask=m, decoder_input_ids=dii)

                output_ids=outputs.logits[:, n_tokens:, :]
                max_pred_ids=outputs.logits[:, n_tokens:, :].argmax(-1).detach().cpu().numpy().tolist()
                
                for i, (max_pred_ids_unbatch, output_ids_unbatch) in enumerate(zip(max_pred_ids, output_ids)):
                    max_pred_ids_unbatch=np.array(max_pred_ids_unbatch)
                    # output_ids[i]=output_ids_unbatch[max_pred_ids_unbatch!=0]
                    max_pred_ids[i]=max_pred_ids_unbatch[max_pred_ids_unbatch!=0].tolist()
                
#                 print("tokenizer.batch_decode(max_pred_ids)", tokenizer.batch_decode(max_pred_ids))
                output_max_pred.extend(tokenizer.batch_decode(max_pred_ids))
                # print("output_max_pred", output_max_pred)

                loss = outputs.loss
                loss_value = float(loss.detach().cpu().numpy().tolist()) / batch_size
                test_losses.append(loss_value)

                pred_labels = outputs['logits'][:, :, :].argmax(-1).detach().cpu().numpy().tolist()
                all_pred_labels += pred_labels
                pbar.set_description(f'dev: loss={np.mean(test_losses):.4f}')
                
    return output_max_pred

In [25]:
output_max_pred=eval_with_prompt(model, tokenizer, device, batch_size, n_tokens,
                     val_input=test_sensor_data, val_label=test_templates, optimizer=optimizer, ce_loss=ce_loss, use_ce_loss=False)
print(len(output_max_pred), len(output_max_pred[0]))

dev: loss=0.0464: 100%|██████████| 21/21 [00:09<00:00,  2.22it/s]

660 135


In [26]:
print(len(output_max_pred), output_max_pred[0])
print(len(test_sensor_data), test_sensor_data[0])
print(len(test_templates), test_templates[0])

660 (0:05) The drone is facing the risk of physical damage. It's a too close to drone at a distance at 0.5 0.5 the pilot is not experienced
660 {"time_stamp": "0:05", "name": NaN, "Distance": 0.5}
660 (0:05) The drone is facing the risk of physical damage. It's flying too close to nan at a distance of 0.5m, and the pilot is not experienced.


In [27]:
# predictions: An iterator over tokenized predictions. Each prediction is a list.
# [
#     ["michael", "dahlquist", "(", "december", "22", ",", "1965", "--", "july",
#      "14", ",", "2005", ")", "was", "a", "drummer", "in", "the", "california",
#      "band", "grateful", "dead", "."],
# ]
def __max_pred_as_generation(output_max_pred):
    result=[]
    for line in output_max_pred:
        tmp=[line.lower().split()]
        result.append(tmp)
        # print("__max_pred_as_generation_reader", tmp)
        
    return result
        
# references: An iterator over lists of tokenized references. Each prediction can have multiple references.
# [[
#     "michael", "dahlquist", "(", "december", "22", ",", "1965", "--", "july",
#     "14", ",", "2005", ")", "was", "a", "drummer", "in", "the", "seattle",
#     "band", "silkworm", "."
# ]]
def __template_as_refer(test_templates):
    result=[]
    for line in test_templates:
        tmp=line.lower().split()
        result.append(tmp)
        # print("__template_as_refer_reader", tmp)
        
    return result
        
# tables: An iterator over the tables
# TEST_TABLE = [
#     [["birth", "name"], ["michael", "dahlquist"]],
#     [["born"], ["december", "22", ",", "1965"]],
#     [["birth", "place"], ["seattle", ",", "washington"]],
#     [["died"], ["july", "14", ",", "2005"]],
#     [["death", "place"], ["skokie", ",", "illinois"]],
#     [["genres"], ["male"]],
#     [["occupation(s)"], ["drummer"]],
#     [["instruments"], ["drums"]],
# ]
def __sensor_data_as_table(test_sensor_data):
    result=[]
    for line in test_sensor_data:
        related_sensor_data=json.loads(line)
        # print("__sensor_data_as_table_reader related_sensor_data", related_sensor_data)
        
        tmp=[]
        for key, value in related_sensor_data.items():
            key_tokens=key.lower().split()
            value_tokens=str(value).lower().split()
            
#             print("__sensor_data_as_table_reader key", key)
#             print("__sensor_data_as_table_reader key_tokens", key_tokens)
            
#             print("__sensor_data_as_table_reader value", value)
#             print("__sensor_data_as_table_reader value_tokens", value_tokens)
            
            tmp.append([key_tokens, value_tokens])
        
        result.append(tmp)
        # print("__sensor_data_as_table_reader", tmp)
    
    return result

In [28]:
generation_it = __max_pred_as_generation(output_max_pred)
reference_it = __template_as_refer(test_templates)
table_it = __sensor_data_as_table(test_sensor_data)

print("generation_it", type(generation_it), generation_it[0], generation_it[:3])
print("reference_it", type(reference_it), reference_it[0], reference_it[:3])
print("table_it", type(table_it), table_it[0], table_it[:3])

generation_it <class 'list'> [['(0:05)', 'the', 'drone', 'is', 'facing', 'the', 'risk', 'of', 'physical', 'damage.', "it's", 'a', 'too', 'close', 'to', 'drone', 'at', 'a', 'distance', 'at', '0.5', '0.5', 'the', 'pilot', 'is', 'not', 'experienced']] [[['(0:05)', 'the', 'drone', 'is', 'facing', 'the', 'risk', 'of', 'physical', 'damage.', "it's", 'a', 'too', 'close', 'to', 'drone', 'at', 'a', 'distance', 'at', '0.5', '0.5', 'the', 'pilot', 'is', 'not', 'experienced']], [['(0:02)', 'the', 'drone', 'is', 'facing', 'the', 'risk', 'of', 'physical', 'damage.', 'it', 'is', 'a', 'too', 'in', 'to', 'the', "drone's", 'distance', 'distance', 'at', '0', 'distance', '0m.5.</s></s>']], [['the', 'flying', 'drone', 'is', 'runing', 'out', 'of', 'battery', 'with', 'only%%', 'charge.</s>']]]
reference_it <class 'list'> ['(0:05)', 'the', 'drone', 'is', 'facing', 'the', 'risk', 'of', 'physical', 'damage.', "it's", 'flying', 'too', 'close', 'to', 'nan', 'at', 'a', 'distance', 'of', '0.5m,', 'and', 'the', 'pil

In [29]:
parent_score_li=[]

for pred, refer, table in zip(generation_it, reference_it, table_it):
    # print("pred", type(pred), pred)
    # print("refer", type(refer), refer)
    # print("table", type(table), table)
    _, _, parent_score, _ = parent(pred, [[refer]], [table], lambda_weight=None)
    parent_score_li.append(parent_score)

print("parent_score_li", len(parent_score_li), parent_score_li[0])

parent_score_li 660 0.02038661519328828


In [26]:
entail_method=overlap_probability
precision, recall, f_score, all_f = parent(
    generation_it,
    [reference_it],
    table_it,
    lambda_weight=None,
    smoothing=0.00001,
    entailment_fn=entail_method)

print("precision", precision)
print("recall", recall)
print("f_score", f_score)
print("all_f", all_f)

TypeError: unhashable type: 'list'

In [70]:

TEST_PREDS = [[['(0:05)', 'the', 'drone', 'is', 'facing', 'the', 'risk', 'of', 'physical']], [['(0:02)', 'the', 'drone', 'is', 'facing']], [['the', 'flying', 'drone', 'is', 'runing', 'out', 'of', 'battery']]]
TEST_REF = [['(0:05)', 'the', 'drone', 'is', 'facing', 'the', 'risk', 'of', 'physical', 'damage.', "it's", 'flying', 'too', 'close', 'to', 'nan', 'at', 'a', 'distance', 'of', '0.5m,', 'and', 'the', 'pilot', 'is', 'not', 'experienced.'], ['(0:02)', 'the', 'drone', 'is', 'facing', 'the', 'risk', 'of', 'physical', 'damage.', "it's", 'flying', 'too', 'close', 'to', 'booth', 'at', 'a', 'distance', 'of', '0.5m,', 'and', 'the', 'pilot', 'is', 'not', 'experienced.'], ['the', 'flying', 'drone', 'is', 'runing', 'out', 'of', 'battery', 'with', 'only', '0%', 'charge.']]
TEST_TABLE = [[['time_stamp'], ['0:05']], [['name'], ['nan']], [['distance'], ['0.5']]], [[['time_stamp'], ['0:02']], [['name'], ['booth']], [['distance'], ['0.5']]], [[['battery_level'], ['0']]]

# _, _, parent_score, _=parent(TEST_PREDS, [TEST_REF], [TEST_TABLE], lambda_weight=None)
# print(parent_score)

def test_parent():
    for pred, score in zip(TEST_PREDS, TEST_SCORES):
      print("pred", type(pred), pred)
      print("prediction = %s" % " ".join(pred[0]))
      _, _, parent_score, _ = parent(pred, [TEST_REF],
                                                     [TEST_TABLE],
                                                     lambda_weight=None)
      print(parent_score)
      

test_parent()

pred <class 'list'> [['(0:05)', 'the', 'drone', 'is', 'facing', 'the', 'risk', 'of', 'physical']]
prediction = (0:05) the drone is facing the risk of physical


ValueError: not enough values to unpack (expected 3, got 1)

In [67]:

TEST_PREDS = [
    ["michael", "dahlquist", "(", "december", "22", ",", "1965", "--", "july",
     "14", ",", "2005", ")", "was", "a", "drummer", "in", "the", "california",
     "band", "grateful", "dead", "."],
]

TEST_REF = [
    "michael", "dahlquist", "(", "december", "22", ",", "1965", "--", "july",
    "14", ",", "2005", ")", "was", "a", "drummer", "in", "the", "seattle",
    "band", "silkworm", "."
]

TEST_TABLE = [
    [["birth", "name"], ["michael", "dahlquist"]],
    [["born"], ["december", "22", ",", "1965"]],
    [["birth", "place"], ["seattle", ",", "washington"]],
    [["died"], ["july", "14", ",", "2005"]],
    [["death", "place"], ["skokie", ",", "illinois"]],
    [["genres"], ["male"]],
    [["occupation(s)"], ["drummer"]],
    [["instruments"], ["drums"]],
]
TEST_SCORES = [0.772, 0.836, 0.848]



In [68]:
_, _, parent_score, _=parent(TEST_PREDS, [TEST_REF],TEST_TABLE,lambda_weight=None)
print(parent_score)

0.08476999753365962


In [69]:
def test_parent():
    for pred, score in zip(TEST_PREDS, TEST_SCORES):
      
      print("prediction = %s" % " ".join(pred))
        
      print("[pred]", type([pred]), [pred])
      print("[[TEST_REF]]", type([[TEST_REF]]), [[TEST_REF]])
      print("[TEST_TABLE]", type([TEST_TABLE]), [TEST_TABLE])
      _, _, parent_score, _ = parent([pred], [[TEST_REF]],
                                                     [TEST_TABLE],
                                                     lambda_weight=None)
      print(parent_score)

test_parent()

prediction = michael dahlquist ( december 22 , 1965 -- july 14 , 2005 ) was a drummer in the california band grateful dead .
[pred] <class 'list'> [['michael', 'dahlquist', '(', 'december', '22', ',', '1965', '--', 'july', '14', ',', '2005', ')', 'was', 'a', 'drummer', 'in', 'the', 'california', 'band', 'grateful', 'dead', '.']]
[[TEST_REF]] <class 'list'> [[['michael', 'dahlquist', '(', 'december', '22', ',', '1965', '--', 'july', '14', ',', '2005', ')', 'was', 'a', 'drummer', 'in', 'the', 'seattle', 'band', 'silkworm', '.']]]
[TEST_TABLE] <class 'list'> [[[['birth', 'name'], ['michael', 'dahlquist']], [['born'], ['december', '22', ',', '1965']], [['birth', 'place'], ['seattle', ',', 'washington']], [['died'], ['july', '14', ',', '2005']], [['death', 'place'], ['skokie', ',', 'illinois']], [['genres'], ['male']], [['occupation(s)'], ['drummer']], [['instruments'], ['drums']]]]
0.7717486485694235
